In [2]:
import os
import fitz #PyMuPDF -- reads the text in a more useable format than PyPDF
import re
import json


In [3]:
def get_author(page, relatedString):
    title_page = page.get_text()
    title_list = title_page.splitlines()
    for m, s in enumerate(title_list):
        if relatedString in s:
            author = title_list[m+1]
    return author

In [4]:
def getWords_ySorted(p):
    # sort words by text_y coordinate
    WordTuples = p.get_textpage().extractWORDS()
    WordTuples.sort(key=lambda x: x[3])
    return WordTuples

In [5]:
def extractRemoveTitle(wordTups):
    q = 0
    title_string = []
    for item in wordTups:
        if item[5] == 0:
            title_string.append(item[4])
            q += 1
    title_string = " ".join(word for word in title_string)
    newTuples = wordTups[q:]
    return newTuples, title_string


In [6]:
def reblockAndSort(wordTups, removeHeader=True, removeFooter=True, anticipateTable=True):
    # group words by block num
    blocks = []
    subblock = []
    ia = 0
    tups = wordTups

    if removeHeader:
        noggin = wordTups[0][5]
        tups = [tup for tup in tups if tup[5] != noggin]

    if removeFooter:
        tootsies = wordTups[-1][5]
        tups = [tup for tup in tups if tup[5] != tootsies]
    
    if anticipateTable:
        Dropsies = []
        for ix, tup in enumerate(tups):
            if ix == 0:
                continue
            elif tup[5] in Dropsies:
                continue
            else:
                prevTup = tups[ix-1]
                if (tup[1] == prevTup[1]) and (abs(tup[2] - prevTup[2]) > 40) and (tup[5] != prevTup[5]):
                    Dropsies.append(tup[5])

    for ix, item in enumerate(tups):
        # Sorted by y_text coord.
        if ix == 0:
            subblock.append(item)

        elif item[5] == subblock[ia][5]:
            subblock.append(item)
            ia += 1                             # track subblock
        
        else:                                   # Append subgroups to page block
            blocks.append(subblock)
            ia = 0
            subblock = []
            subblock.append(item)
            
    blocks.append(subblock)

    flattenedBlocks = [item for items in blocks for item in items if item[5] not in Dropsies]
    return flattenedBlocks


In [7]:
def ProperSortString(smushedBlox, lineSpaceVal=5):
    final_sort = []
    subfinal_sort = []
    for ix, item in enumerate(smushedBlox):
        
        if ix == 0:
            subfinal_sort.append(item)

        elif item[3] == smushedBlox[ix-1][3]:
            subfinal_sort.append(item)
        
        elif abs(item[3] - smushedBlox[ix-1][3]) <= lineSpaceVal:
            subfinal_sort.append(item)

        else:
            subfinal_sort.sort(key = lambda x: x[0])
            final_sort.append(subfinal_sort)
            subfinal_sort = []
            subfinal_sort.append(item)
    
    subfinal_sort.sort(key= lambda x: x[0])
    final_sort.append(subfinal_sort)

    flattenFinalSort = [item[4] for items in final_sort for item in items]

    return flattenFinalSort

In [8]:
# returns dictionary with author, title, and full string. Headers and Footers are removed, while citations and bibliography is left in
def pdfToDict(pdf):
    Strings = []
    dict_ = {}
    with fitz.open(pdf) as file:
        for i, page in enumerate(file):
            if i == 0:
                dict_["author"] = get_author(page, "by the author")
            
            else:
                wordTuples = getWords_ySorted(page)
                if i == 1:
                    wordTuples, dict_["title"] = extractRemoveTitle(wordTuples)
                
                blockedAndSorted = reblockAndSort(wordTuples)
                sortedStringy = ProperSortString(blockedAndSorted)
                strCheese = ' '.join(item for item in sortedStringy)
                Strings.append(strCheese)

    dict_["content"] = ' '.join(item for item in Strings)
    return dict_

In [9]:
Str_ = []
with fitz.open("metaepistemology_sc.pdf") as file: #Ugh damn tables are gonna crush me
    page = file[23]
    wordTuples = getWords_ySorted(page)
    blockedAndSorted = reblockAndSort(wordTuples)
    # sortedStringy = ProperSortString(blockedAndSorted)
    # strCheese = ' '.join(item for item in sortedStringy)
    Str_.append(blockedAndSorted)

for items in Str_:
    for item in items:
        print(item)

(110.62035369873047, 77.96035766601562, 179.83908081054688, 88.51972198486328, 'Coarse-Grained', 0, 1, 0)
(216.741943359375, 77.96035766601562, 272.4552917480469, 88.51972198486328, 'Fine-grained', 0, 2, 0)
(110.62035369873047, 98.55117797851562, 147.5615692138672, 109.11054229736328, 'Informal', 1, 0, 1)
(150.20108032226562, 98.55117797851562, 171.90267944335938, 109.11054229736328, 'work', 1, 0, 2)
(174.5425262451172, 98.55117797851562, 185.1018829345703, 109.11054229736328, 'on', 1, 0, 3)
(67.32696533203125, 99.07949829101562, 104.28507232666016, 109.63886260986328, 'Informal', 1, 0, 0)
(110.62035369873047, 110.69473266601562, 134.65567016601562, 121.25409698486328, 'belief', 2, 0, 0)
(137.29542541503906, 110.69473266601562, 152.54103088378906, 121.25409698486328, 'and', 2, 0, 1)
(155.18087768554688, 110.69473266601562, 165.1569366455078, 121.25409698486328, 'its', 2, 0, 2)
(167.7971954345703, 110.69473266601562, 201.81524658203125, 121.25409698486328, 'revision', 2, 0, 3)
(110.6203

In [129]:
Str_ = []
with fitz.open("medicine_sc.pdf") as file:
    page = file[20]
    wordTuples = getWords_ySorted(page)
    blockedAndSorted = reblockAndSort(wordTuples)
    # sortedStringy = ProperSortString(blockedAndSorted)
    # strCheese = ' '.join(item for item in sortedStringy)
    Str_.append(blockedAndSorted)

for items in Str_:
    for item in items:
        print(item)

(40.40059280395508, 73.21150207519531, 51.4879264831543, 87.9946060180664, '4.', 0, 0, 0)
(55.18370056152344, 73.21150207519531, 137.30433654785156, 87.9946060180664, 'Reductionism', 0, 0, 1)
(140.9996337890625, 73.21150207519531, 162.34349060058594, 87.9946060180664, 'and', 0, 0, 2)
(166.0392608642578, 73.21150207519531, 209.57302856445312, 87.9946060180664, 'Holism', 0, 0, 3)
(213.26832580566406, 73.21150207519531, 224.76661682128906, 87.9946060180664, 'in', 0, 0, 4)
(228.46238708496094, 73.21150207519531, 284.28704833984375, 87.9946060180664, 'Medicine', 0, 0, 5)
(40.40059280395508, 103.30606079101562, 52.13584899902344, 113.86542510986328, 'As', 1, 0, 0)
(56.74437713623047, 103.30606079101562, 64.95745086669922, 113.86542510986328, 'in', 1, 0, 1)
(69.5655517578125, 103.30606079101562, 93.02423095703125, 113.86542510986328, 'many', 1, 0, 2)
(97.63233947753906, 103.30606079101562, 123.1543197631836, 113.86542510986328, 'fields,', 1, 0, 3)
(127.76242065429688, 103.30606079101562, 159.

In [113]:
# Sure, there are plenty of libraries to help convert characters for tokens. But I wanted to do it.
def removeProblemChars(string, charsToDrop, charsToReplace):
    string_cleaning = string
    for ix, chars in enumerate(charsToDrop):
        string_cleaning = string_cleaning.replace(chars, charsToReplace[ix])
    return string_cleaning

In [116]:
pdfDict = pdfToDict("medicine_sc.pdf")
print(pdfDict["content"])

Philosophy of medicine is a field that seeks to explore fundamental issues in theory, research, and practice within the health sciences, particularly metaphysical and epistemological topics. Its historic roots arguably date back to ancient times, to the Hippocratic corpus among other sources, and there have been extended scholarly discussions on key concepts in the philosophy of medicine since at least the 1800s. Debates have occurred in the past over whether there is a distinct field rightly termed “philosophy of medicine” (e.g., Caplan 1992) but as there are now dedicated journals and professional organizations, a relatively well-established canon of scholarly literature, and distinctive questions and problems, it is defensible to claim that philosophy of medicine has now established itself. Although ethics and values are part of many problems addressed within the philosophy of medicine, bioethics is generally considered to be a distinct field, and hence is not explored in this entry

In [121]:
bikerTassels = [
    "[", "]", '“', '”', " ,", "- ", " .", " -",
    "(Zizzo 2010, 2013, Winking and Mizer 2013. "
]
primString = [
    "", "", '"', '"', ",", "-", ".", "-",
    "(Zizzo 2010, 2013, Winking and Mizer 2013). "  # This dude caused me so much pain (see Eye Strain, Knucklehead 2023)
]


pdfDict["content"] = removeProblemChars(pdfDict["content"], bikerTassels, primString)

In [122]:
print(pdfDict["content"])

Philosophy of medicine is a field that seeks to explore fundamental issues in theory, research, and practice within the health sciences, particularly metaphysical and epistemological topics. Its historic roots arguably date back to ancient times, to the Hippocratic corpus among other sources, and there have been extended scholarly discussions on key concepts in the philosophy of medicine since at least the 1800s. Debates have occurred in the past over whether there is a distinct field rightly termed "philosophy of medicine" (e.g., Caplan 1992) but as there are now dedicated journals and professional organizations, a relatively well-established canon of scholarly literature, and distinctive questions and problems, it is defensible to claim that philosophy of medicine has now established itself. Although ethics and values are part of many problems addressed within the philosophy of medicine, bioethics is generally considered to be a distinct field, and hence is not explored in this entry